In [7]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, ConfusionMatrixDisplay, RocCurveDisplay, confusion_matrix, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import OneHotEncoder
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification

In [15]:
df = pd.read_csv("../retouren.csv")

In [16]:
df["user_dob"] = pd.to_datetime(df.user_dob)
df["user_reg_date"] = pd.to_datetime(df.user_reg_date)
df["order_date"] = pd.to_datetime(df.order_date)
df["delivery_date"] = pd.to_datetime(df.delivery_date)

# Löschen der Datensätze mit Preis von 999
df = df.drop(df[df.item_price == 999].index)

# Löschen der Datensätze wo das Lieferdatum kleiner dem Bestelldatum
df = df.drop(df[df.delivery_date < df.order_date].index)

# Anpassen der Dateninkonsistenzen bei item_color
df = df.drop(df[df.item_color == "?"].index)
df.loc[df.item_color == "brwon","item_color"] = "brown"
df.loc[df.item_color == "blau","item_color"] = "blue"
df.loc[df.item_color == "dark oliv","item_color"] = "dark olive"

# Löschen der Datensätze ohne delivery_date
df.drop(df[df.delivery_date.isna()].index, inplace=True)

In [17]:
# age
df["age"] = df.order_date.dt.year - df.user_dob.dt.year -((df.order_date.dt.date)<(df.user_dob.dt.date))

# age bins
df.loc[df.age > 100, "age"] = df.loc[df.age > 100, "age"] - 100
bins=[0,10,20,30,40,50,60,70,80,90,100]
df["age"] = pd.cut(df.age,bins=bins)

# acc_age
df["acc_age"] = df.order_date - df.user_reg_date 
df["acc_age"] = df.acc_age.dt.days

# maxprice and markdown
df_mark = df.groupby(["item_id","item_size","item_color"]).agg(maxprice=("item_price","max")).reset_index()
df = pd.merge(df,df_mark)
df["markdown"] = 0
df.loc[df.item_price < df.maxprice,"markdown"] = 1

# order_id und order_size
order_id_generator = df.groupby(['order_date', 'user_id'], sort= False).size().reset_index(name='order_size')
order_id_generator['order_id'] = order_id_generator.index
df = pd.merge(df,order_id_generator,on=['user_id', 'order_date'], how='left')

# return_ratio 
df_return_ratio = df.groupby("item_id").agg(returned = ("return", "sum"), item_order= ("return", "count")).reset_index()
df_return_ratio["not_returned"] = df_return_ratio.item_order - df_return_ratio.returned
df_return_ratio["return_ratio"] = df_return_ratio.returned / df_return_ratio.item_order
df_return_ratio.drop(["returned","item_order","not_returned"],axis=1,inplace=True)
df = pd.merge(df,df_return_ratio,on="item_id",how="left")

# user_return_ratio
df_user_return = df.groupby("user_id").agg(user_return= ("return", "sum"), user_orders= ("return", "count")).reset_index()
df_user_return["user_return_ratio"] = df_user_return.user_return / df_user_return.user_orders
df_user_return.drop(["user_return", "user_orders"], axis=1, inplace= True)
df = pd.merge(df, df_user_return, on= "user_id", how= "left")

# nth_order_of_user
df_nth_order_of_user = df.groupby(['user_id', 'order_id'], as_index=False).size().drop('size', axis=1)
df_nth_order_of_user['nth_order_of_user'] = df_nth_order_of_user.groupby(['user_id']).cumcount()+1
df = pd.merge(df,df_nth_order_of_user,on=['user_id', 'order_id'], how='left')

# item_id_>1_in_order
item_counts_in_order = df.groupby(['order_id', 'item_id'], as_index=False).size()
item_counts_in_order['item_id_>1_in_order'] = item_counts_in_order['size'] > 1
df = pd.merge(df,item_counts_in_order[['order_id', 'item_id', 'item_id_>1_in_order']],on=['order_id', 'item_id'], how='left')

# item_id_ordered_previously
df['item_id_ordered_previously'] = False
for index, row in df.iterrows():
    item_id = row['item_id']
    order_id = row['order_id']
    user_id = row['user_id']
    for i2, prev_row in df[df.user_id == user_id].iterrows():
        if prev_row['order_id'] == order_id: 
            break
        if prev_row['item_id'] == item_id: 
            df.loc[index, 'item_id_ordered_previously'] = True
            break

In [19]:
df.columns

Index(['order_item_id', 'order_date', 'delivery_date', 'item_id', 'item_size',
       'item_color', 'brand_id', 'item_price', 'user_id', 'user_title',
       'user_dob', 'user_state', 'user_reg_date', 'return', 'age', 'acc_age',
       'maxprice', 'markdown', 'order_size', 'order_id', 'return_ratio',
       'user_return_ratio', 'nth_order_of_user', 'item_id_>1_in_order',
       'item_id_ordered_previously'],
      dtype='object')

In [14]:
enc = OneHotEncoder(handle_unknown='ignore',sparse = False) 
df_1 = enc.fit_transform(df[[ "item_color",	"user_title","user_state", "age", "item_size","item_id_>1_in_order" ]])
e = enc.categories_
a = []
for i in range(6):
    a = np.append(a,e[i])
a = [str(i) for i in a]
df_1 = pd.DataFrame(df_1,columns=a)
df = pd.concat([df,df_1],axis=1)
df.drop([ "item_color",	"user_title",	"user_state", "age", "item_size","item_id_>1_in_order" ],axis=1,inplace=True)

KeyError: "None of [Index(['item_color', 'user_title', 'user_state', 'age', 'item_size',\n       'item_id_>1_in_order'],\n      dtype='object')] are in the [columns]"

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('return', axis=1), df['return'])

In [13]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[int64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[

In [11]:
LogReg.score(X_test, y_test)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[int64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[